# Run Agent
- Implement a workflow to write a daily AI newsletter
- see README.md for details



```mermaid
  flowchart TD
      Start([Start Newsletter Workflow]) --> Step1[1: Gather URLs<br/>RSS + HTML + REST APIs<br/>17+ sources]

      Step1 --> Step2[2: Filter URLs<br/>Dedupe Previously Seen<br/>LLM AI Filter<br/>]

      Step2 --> Step3[3: Download AI Articles<br/>Extract Text]

      Step3 --> Step4[4: LLM Summarization<br/>Bullet-point summaries]

      Step4 --> Step5[5: Cluster by Topic<br/>HDBSCAN + Embeddings]

      Step5 --> Step6[6: Rate Articles<br/>Identify<br/>On-Topic, Quality<br/>Using Elo 1-1 Comparisons]

      Step6 --> Step7[7: Select Sections<br/>Top Articles by Topic<br/>Newsletter Outline]

      Step7 --> Step8[8: Draft Sections<br/>Write Headlines & Content<br/>Run Critic-Optimizer Loop]

      Step8 --> Critique8[Section Critique<br/>Quality Evaluation]
      Critique8 --> Check8{Score >= 8.0?}
      Check8 -->|No & iter < 3| Improve8[Apply Improvements]
      Improve8 --> Critique8
      Check8 -->|Yes or iter = 3| Step9[9: Finalize Newsletter<br/>Generate Title<br/>Assemble Sections]

      Step9 --> Critique9[Full Newsletter<br/>Critic-Optimizer Loop<br/>Evaluate Quality<br/>5 dimension scores]

      Critique9 --> Check9{Score >= 8.0<br/>OR<br/>iter = 3?}

      Check9 -->|No| Optimizer[Optimizer Agent<br/>Apply Recommendations<br/>Improve Newsletter]
      Optimizer --> Critique9

      Check9 -->|Yes| Final([Publication-Ready<br>Newsletter<br/>Quality Score >= 8.0])

      style Step1 fill:#e1f5ff
      style Step2 fill:#e1f5ff
      style Step3 fill:#e1f5ff
      style Step4 fill:#fff9c4
      style Step5 fill:#fff9c4
      style Step6 fill:#fff9c4
      style Step7 fill:#e8f5e9
      style Step8 fill:#e8f5e9
      style Critique8 fill:#ffe0e0
      style Check8 fill:#ffe0e0
      style Improve8 fill:#ffe0e0
      style Step9 fill:#f3e5f5
      style Critique9 fill:#f3e5f5
      style Check9 fill:#f3e5f5
      style Optimizer fill:#f3e5f5
  ```

In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable, Text
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url, AgentState

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            2.6.0
OpenAI Agents SDK  0.4.0
Pydantic           2.12.3


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger
debuglevel = logging.DEBUG

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig()
    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(debuglevel)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(debuglevel)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(debuglevel)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.debug("Test debug message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

07:48:54 | NewsletterAgent.newsletter_agent | DEBUG | Test debug message
07:48:54 | NewsletterAgent.newsletter_agent | INFO | Test info message
07:48:54 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
07:48:54 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

# Run Agent Worfklow

In [5]:
conn = sqlite3.connect("newsletter_agent.db")
AgentState.list_sessions(conn, n_records=10)


['test_newsletter_20251022174713041313',
 'test_newsletter_20251022074726590734',
 'test_newsletter_20251021183200598515',
 'test_newsletter_20251021090026683559',
 'test_newsletter_20251021085814698013',
 'test_newsletter_20251021085107930253',
 'test_newsletter_20251021084448443269',
 'test_newsletter_20251021075556415101',
 'test_newsletter_20251020165646176918',
 'test_newsletter_20251020085022924065']

In [6]:
z = AgentState.get_latest_by_session(conn, "test_newsletter_20251021090026683559")
z.step_name

'rate_articles'

In [7]:
print("🚀 Creating NewsletterAgent...")

do_download=True
reprocess_since=None
# reprocess_since='2025-10-21 06:00:00'

try:
    # set up state
    session_id = 'test_newsletter_20251021090026683559'
    step_name = 'filter_urls'
    del session_id, step_names
except Exception as e:
    print(e)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 reprocess_since=reprocess_since,
                                 verbose=False
                                )
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 reprocess_since=reprocess_since,
                                 verbose=False
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    state.serialize_to_db("initialize")

🚀 Creating NewsletterAgent...
name 'step_names' is not defined
session_id is not defined
test_newsletter_20251023074854730516


In [8]:
agent.state.get_status()


{'headlines': {'total': 0},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'gather_urls',
  'workflow_complete': False,
  'progress_percentage': 0.0,
  'completed_steps': 0,
  'total_steps': 9,
  'max_edits': 2,
  'concurrency': 16,
  'do_download': True,
  'reprocess_since': None},
 'processing': {'topic_clusters': 0,
  'newsletter_sections': 0,
  'final_newsletter_length': 0}}

In [9]:
agent.state.headline_df


""


In [10]:
for step in agent.state.steps:
    print(step)
    print("---")


⚪ Gather URLs [not_started]
   
---
⚪ Filter URLs [not_started]
   
---
⚪ Download Articles [not_started]
   
---
⚪ Extract Summaries [not_started]
   
---
⚪ Rate Articles [not_started]
   
---
⚪ Cluster By Topic [not_started]
   
---
⚪ Select Sections [not_started]
   
---
⚪ Draft Sections [not_started]
   
---
⚪ Finalize Newsletter [not_started]
   
---


In [11]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


07:48:58 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Starting check_workflow_status
07:48:58 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Completed check_workflow_status


⏱️  Total execution time: 13.29s
📊 Final result:
Current workflow status (from persistent state):

- Progress: 0.0% (0/9 complete)
- Status summary: 0 complete, 0 started, 0 failed, 9 not started
- Next step: Step 0 — Gather URLs

Step details:
- Step 0: Gather URLs — not_started
- Step 1: Filter URLs — not_started
- Step 2: Download Articles — not_started
- Step 3: Extract Summaries — not_started
- Step 4: Rate Articles — not_started
- Step 5: Cluster By Topic — not_started
- Step 6: Select Sections — not_started
- Step 7: Draft Sections — not_started
- Step 8: Finalize Newsletter — not_started

What would you like me to do next?
- Start the full workflow (run all remaining steps in sequence), or
- Run a specific step (tell me which), or
- Inspect internal state for debugging.


In [12]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


07:49:15 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Starting check_workflow_status
07:49:15 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Completed check_workflow_status
07:49:18 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Starting Step 1: Gather URLs
07:49:18 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Cleaning out download/sources: 
07:49:18 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Successfully cleaned out download/sources
07:49:18 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Cleaning out download/text: 
07:49:18 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Successfully cleaned out download/text
07:49:18 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Cleaning out download/html: 
07:49:18 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Successfully cleaned out download/html
2025-10-23 07:49:18,197 - fetcher_5629869

▶ Starting Step 1: gather_urls


2025-10-23 07:49:18,406 - fetcher_5629869328 - INFO - [fetch_rss] RSS fetch successful for HackerNoon: 50 articles
2025-10-23 07:49:18,407 - fetcher_5629869328 - INFO - [fetch_html] Fetching HTML from Techmeme: https://www.techmeme.com/river
2025-10-23 07:49:18,504 - fetcher_5629869328 - INFO - [fetch_rss] RSS fetch successful for Ars Technica: 20 articles
2025-10-23 07:49:18,505 - fetcher_5629869328 - INFO - [fetch_rss] Fetching RSS from The Register: https://www.theregister.com/software/ai_ml/headlines.atom
2025-10-23 07:49:18,592 - fetcher_5629869328 - INFO - [fetch_rss] RSS fetch successful for Hacker News: 30 articles
2025-10-23 07:49:18,593 - fetcher_5629869328 - INFO - [fetch_html] Fetching HTML from The Verge: https://www.theverge.com/ai-artificial-intelligence
2025-10-23 07:49:18,608 - fetcher_5629869328 - INFO - [fetch_rss] RSS fetch successful for The Register: 50 articles
2025-10-23 07:49:18,609 - fetcher_5629869328 - INFO - [fetch_html] Fetching HTML from VentureBeat: http

,source,url
0,Ars Technica,20
1,Bloomberg,28
2,Business Insider,16
3,FT,47
4,Feedly AI,70
5,Hacker News,30
6,HackerNoon,50
7,New York Times,23
8,NewsAPI,96
9,Reddit,51


,source,title,url,published,rss_summary,id
0,Ars Technica,We let OpenAI’s “Agent Mode” surf the web for ...,https://arstechnica.com/features/2025/10/we-le...,"Thu, 23 Oct 2025 11:00:08 +0000","From scanning emails to building fansites, Atl...",0
1,Ars Technica,General Motors will integrate AI into its cars...,https://arstechnica.com/cars/2025/10/ai-and-ha...,"Wed, 22 Oct 2025 20:29:11 +0000",Do we want LLMs in our cars? GM thinks we do.,1
2,Ars Technica,When sycophancy and bias meet medicine,https://arstechnica.com/health/2025/10/when-sy...,"Wed, 22 Oct 2025 19:46:01 +0000","Biased, eager-to-please models threaten health...",2
3,Ars Technica,OpenAI looks for its “Google Chrome” moment wi...,https://arstechnica.com/ai/2025/10/openais-new...,"Tue, 21 Oct 2025 19:02:10 +0000","MacOS version launches today, includes Agent M...",3
4,Ars Technica,YouTube’s likeness detection has arrived to he...,https://arstechnica.com/google/2025/10/youtube...,"Tue, 21 Oct 2025 18:46:42 +0000",Likeness detection will flag possible AI fakes...,4
...,...,...,...,...,...,...
730,NewsAPI,Construction Toys Market Forecast Report 2025-...,https://www.globenewswire.com/news-release/202...,2025-10-22T10:25:00Z,NaN,730
731,NewsAPI,The Chrome Killer? ChatGPT Maker Takes On Goog...,https://www.huffpost.com/entry/openai-chatgpt-...,2025-10-22T10:25:47Z,NaN,731
732,NewsAPI,Construction & Building Materials Market Outlo...,https://www.globenewswire.com/news-release/202...,2025-10-22T11:06:00Z,NaN,732
733,NewsAPI,"Anthropic, Google in talks on multibillion-dol...",https://www.irishtimes.com/business/2025/10/22...,2025-10-22T10:55:54Z,NaN,733


07:49:50 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Completed Step 1: Gathered 853 articles
07:49:52 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Starting check_workflow_status
07:49:52 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Completed check_workflow_status


⏱️  Total execution time: 49.45s
📊 Final result:
Step 1 (Gather URLs) has been run.

Summary:
- Ran workflow status check (pre-run).
- Executed fetch_urls (Step 1/gather URLs).
- Result: Completed successfully — gathered 853 articles from 17 sources; 735 articles stored in persistent state.
- Checked workflow status after the step.

Current workflow status:
- Progress: 11.1% (1/9 complete)
- Next step: Step 1 — Filter URLs (filter to AI-related content)
- Totals in state: 735 articles stored; 0 AI-related flagged yet.

What would you like next?
- Proceed with Step 2 (filter URLs to AI-related content), or
- Inspect stored articles, or
- Run a different step.


In [13]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
10,Techmeme,158
8,NewsAPI,96
4,Feedly AI,70
9,Reddit,51
12,The Register,50
6,HackerNoon,50
3,FT,47
5,Hacker News,30
1,Bloomberg,28
13,The Verge,26


In [14]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


07:49:57 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Starting Step 2: Filter URLs
07:49:57 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | 🔍 Filtering 735 headlines...
07:49:57 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | 🔄 Checking for duplicates (all urls without date restrictions)
07:49:57 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | 🔍 Filtering 735 articles for dupes.


▶ Starting Step 2: filter_urls
new - keeping https://arstechnica.com/features/2025/10/we-let-openais-agent-mode-surf-the-web-for-us-heres-what-happened/
found with no cutoff set - ignoring as duplicate https://arstechnica.com/cars/2025/10/ai-and-hands-free-driving-are-coming-to-gms-vehicles/
found with no cutoff set - ignoring as duplicate https://arstechnica.com/health/2025/10/when-sycophancy-and-bias-meet-medicine/
found with no cutoff set - ignoring as duplicate https://arstechnica.com/ai/2025/10/openais-new-atlas-web-browser-wants-to-let-you-chat-with-a-page/
found with no cutoff set - ignoring as duplicate https://arstechnica.com/google/2025/10/youtube-rolls-out-likeness-detection-to-help-creators-combat-ai-fakes/
found with no cutoff set - ignoring as duplicate https://arstechnica.com/ai/2025/10/claude-code-gets-a-web-version-but-its-the-new-sandboxing-that-really-matters/
found with no cutoff set - ignoring as duplicate https://arstechnica.com/features/2025/10/should-an-ai-copy-

07:49:57 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | URL deduplication: 467 duplicates filtered, 268 new URLs remain
07:49:57 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | 🔍 Filtering 735 headlines for AI relevance using LLM...
07:49:57 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Initialized LangfuseClient


new - keeping https://www.reuters.com/world/china/nexperias-china-unit-resumes-chip-sales-domestic-distributors-sources-say-2025-10-23/
new - keeping https://www.wsj.com/tech/ai/ai-race-tech-workers-schedule-1ea9a116
new - keeping https://therecord.media/spacex-disables-starlink-kits-in-myanmar-scam-compounds
new - keeping https://x.com/cryps1s/status/1981037851279278414
new - keeping https://www.cnbc.com/2025/10/22/elon-musk-tesla-ai5-nvidia.html
new - keeping https://www.wsj.com/business/entrepreneurship/trump-administration-in-talks-to-take-equity-stakes-in-quantum-computing-firms-60ee5143
new - keeping https://www.bloomberg.com/news/articles/2025-10-23/polymarket-is-seeking-funding-at-a-valuation-of-up-to-15-billion
new - keeping https://www.bloomberg.com/news/articles/2025-10-22/crypto-exchange-htx-sued-by-uk-for-unlawful-promotion-of-assets
new - keeping https://www.geekwire.com/2025/amazon-unveils-ai-powered-augmented-reality-glasses-for-delivery-drivers/
found with no cutoff se

07:49:57 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
07:49:57 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Parsed prompt 'newsagent/filter_urls': model=gpt-5-nano, reasoning_effort=medium, system_len=459, user_len=954
07:49:57 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Processing 11 chunks with concurrency 16
07:50:58 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Completed Step 2: 122 AI-related articles, 467 duplicates removed


⏱️  Total execution time: 61.01s
📊 Final result:
✅ Step 2 filter_urls completed successfully! Removed 467 duplicate URLs, classified 268 new articles, found 122 AI-related.


In [15]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


07:50:58 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Starting Step 3: Download Articles
07:50:58 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Starting concurrent scraping of 122 AI-related articles
07:50:58 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Launching browser for 122 URLs with 16 concurrent workers
07:50:58 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Worker 0 fetching 1 of 122 https://biztoc.com/x/fc79d5e56ee57ccb


▶ Starting Step 3: download_articles


07:50:58 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | scrape_url(https://biztoc.com/x/fc79d5e56ee57ccb)
07:50:58 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | scraping https://biztoc.com/x/fc79d5e56ee57ccb to download/html
07:50:58 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Downloading https://biztoc.com/x/fc79d5e56ee57ccb
07:50:58 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Worker 1 fetching 2 of 122 https://financialpost.com/pmn/business-pmn/tesla-stops-growing-so-investors-pivot-to-dreams-of-an-ai-future
07:50:58 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | scrape_url(https://financialpost.com/pmn/business-pmn/tesla-stops-growing-so-investors-pivot-to-dreams-of-an-ai-future)
07:50:58 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | scraping https://financialpost.com/pmn/business-pmn/tesla-stops-growing-so-investors-pivot-to-dreams-of-an-ai-future to download/html
07:50:58 

Starting with 118 rows...
Processing 118 files...
Reading and truncating files to 8192 tokens using text-embedding-3-large tokenizer...
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Getting embeddings for 107 texts...
Creating indexed similarity matrix...
Finding pairs with similarity > 0.925...
Filtering dataframe...
  Pair: 43 vs 59
    43: Feedly AI - Smart Attacks, Smarter Defenses: How AI is Transforming DDoS Attacks
    

,title,status,final_url
64,App that translates speech to sign language in real-time wins top innovation prize,skipped,https://www.cnn.com/world/africa/speech-to-sign-language-app-wins-africa-innovation-prize-spc
65,Reddit sues Perplexity for scraping data to train AI system,401,https://www.reuters.com/world/reddit-sues-perplexity-scraping-data-train-ai-system-2025-10-22/
1,"Nokia Shares Surge Most Since 2021 as AI, Cloud Boost Profit",skipped,https://www.bloomberg.com/news/articles/2025-10-23/nokia-profit-surges-past-estimates-on-network-growth
114,India proposes strict IT rules for labelling deepfakes amid AI misuse,404,https://www.channelnewsasia.com/business/india-proposes-strict-it-rules-labelling-deepfakes-amid-ai-misuse-5417516
25,Watch Your Wallet: ChatGPT Atlas Might Try to Buy the Wrong Product for You,None,https://uk.pcmag.com/ai/160865/watch-your-wallet-chatgpt-atlas-might-try-to-buy-the-wrong-product-for-you
36,"AI creative studio Wonder Studios, backed by OpenAI and Google DeepMind execs, raised a $12M seed led by Atomico and aims to release its first content in 2026 (Najat Kantouar/Wall Street Journal)",skipped,https://www.wsj.com/tech/ai/openai-backed-creative-studio-raises-12-million-as-it-eyes-ai-content-drive-cad81e1f
4,Walmart Reshuffles Key Merchandising Team Amid AI Integration,skipped,https://www.bloomberg.com/news/articles/2025-10-22/walmart-reorganizes-merchandising-team-with-focus-on-technology
3,McKinsey Says Bank Profits Face Possible $170 Billion AI Hit,skipped,https://www.bloomberg.com/news/articles/2025-10-23/mckinsey-says-bank-profits-face-possible-170-billion-ai-hit
2,"Why All the Buzz About AGI? And What Is It Anyway?AI companies are in a race to create artificial general intelligence, seeing it as the catalyst for an “intelligence explosion” that reshapes the economy and society. If only they could agree on what AGI would actually look like, and how it might be achieved",skipped,https://www.bloomberg.com/news/features/2025-10-23/what-is-agi-why-openai-anthropic-are-targeting-artificial-general-intelligence
69,"OpenAI, Oracle, and Vantage Data Centers plan to build a data center in Wisconsin called Lighthouse, costing $15B+ and set to open in 2028, as part of Stargate",401,https://www.reuters.com/business/openai-oracle-announce-stargate-data-center-site-wisconsin-2025-10-22/


07:56:57 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Completed Step 3: Downloaded 111 articles


⏱️  Total execution time: 358.63s
📊 Final result:
✅ Step 3 download_articles completed successfully! Downloaded 111 AI-related articles with 96% success rate.
📊 Average article length: 5631 characters
🔗 Content stored in persistent state.


In [16]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["status"]!=200])




,id,source,title,status,final_url
11,214,HackerNoon,"Smart Attacks, Smarter Defenses: How AI is Transforming DDoS Attacks",exists,https://hackernoon.com/smart-attacks-smarter-defenses-how-ai-is-transforming-ddos-attacks?source=rss
105,321,Reddit,App that translates speech to sign language in real-time wins top innovation prize,skipped,https://www.cnn.com/world/africa/speech-to-sign-language-app-wins-africa-innovation-prize-spc
106,331,Reddit,Reddit sues Perplexity for scraping data to train AI system,401,https://www.reuters.com/world/reddit-sues-perplexity-scraping-data-train-ai-system-2025-10-22/
107,21,Bloomberg,"Nokia Shares Surge Most Since 2021 as AI, Cloud Boost Profit",skipped,https://www.bloomberg.com/news/articles/2025-10-23/nokia-profit-surges-past-estimates-on-network-growth
108,710,NewsAPI,India proposes strict IT rules for labelling deepfakes amid AI misuse,404,https://www.channelnewsasia.com/business/india-proposes-strict-it-rules-labelling-deepfakes-amid-ai-misuse-5417516
109,129,Feedly AI,Watch Your Wallet: ChatGPT Atlas Might Try to Buy the Wrong Product for You,None,https://uk.pcmag.com/ai/160865/watch-your-wallet-chatgpt-atlas-might-try-to-buy-the-wrong-product-for-you
110,155,Feedly AI,"AI creative studio Wonder Studios, backed by OpenAI and Google DeepMind execs, raised a $12M seed led by Atomico and aims to release its first content in 2026 (Najat Kantouar/Wall Street Journal)",skipped,https://www.wsj.com/tech/ai/openai-backed-creative-studio-raises-12-million-as-it-eyes-ai-content-drive-cad81e1f
111,26,Bloomberg,Walmart Reshuffles Key Merchandising Team Amid AI Integration,skipped,https://www.bloomberg.com/news/articles/2025-10-22/walmart-reorganizes-merchandising-team-with-focus-on-technology
112,23,Bloomberg,McKinsey Says Bank Profits Face Possible $170 Billion AI Hit,skipped,https://www.bloomberg.com/news/articles/2025-10-23/mckinsey-says-bank-profits-face-possible-170-billion-ai-hit
113,22,Bloomberg,"Why All the Buzz About AGI? And What Is It Anyway?AI companies are in a race to create artificial general intelligence, seeing it as the catalyst for an “intelligence explosion” that reshapes the economy and society. If only they could agree on what AGI would actually look like, and how it might be achieved",skipped,https://www.bloomberg.com/news/features/2025-10-23/what-is-agi-why-openai-anthropic-are-targeting-artificial-general-intelligence


In [17]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["html_path"]==""])



,id,source,title,status,final_url
105,321,Reddit,App that translates speech to sign language in real-time wins top innovation prize,skipped,https://www.cnn.com/world/africa/speech-to-sign-language-app-wins-africa-innovation-prize-spc
106,331,Reddit,Reddit sues Perplexity for scraping data to train AI system,401,https://www.reuters.com/world/reddit-sues-perplexity-scraping-data-train-ai-system-2025-10-22/
107,21,Bloomberg,"Nokia Shares Surge Most Since 2021 as AI, Cloud Boost Profit",skipped,https://www.bloomberg.com/news/articles/2025-10-23/nokia-profit-surges-past-estimates-on-network-growth
108,710,NewsAPI,India proposes strict IT rules for labelling deepfakes amid AI misuse,404,https://www.channelnewsasia.com/business/india-proposes-strict-it-rules-labelling-deepfakes-amid-ai-misuse-5417516
109,129,Feedly AI,Watch Your Wallet: ChatGPT Atlas Might Try to Buy the Wrong Product for You,None,https://uk.pcmag.com/ai/160865/watch-your-wallet-chatgpt-atlas-might-try-to-buy-the-wrong-product-for-you
110,155,Feedly AI,"AI creative studio Wonder Studios, backed by OpenAI and Google DeepMind execs, raised a $12M seed led by Atomico and aims to release its first content in 2026 (Najat Kantouar/Wall Street Journal)",skipped,https://www.wsj.com/tech/ai/openai-backed-creative-studio-raises-12-million-as-it-eyes-ai-content-drive-cad81e1f
111,26,Bloomberg,Walmart Reshuffles Key Merchandising Team Amid AI Integration,skipped,https://www.bloomberg.com/news/articles/2025-10-22/walmart-reorganizes-merchandising-team-with-focus-on-technology
112,23,Bloomberg,McKinsey Says Bank Profits Face Possible $170 Billion AI Hit,skipped,https://www.bloomberg.com/news/articles/2025-10-23/mckinsey-says-bank-profits-face-possible-170-billion-ai-hit
113,22,Bloomberg,"Why All the Buzz About AGI? And What Is It Anyway?AI companies are in a race to create artificial general intelligence, seeing it as the catalyst for an “intelligence explosion” that reshapes the economy and society. If only they could agree on what AGI would actually look like, and how it might be achieved",skipped,https://www.bloomberg.com/news/features/2025-10-23/what-is-agi-why-openai-anthropic-are-targeting-artificial-general-intelligence
114,343,Techmeme,"OpenAI, Oracle, and Vantage Data Centers plan to build a data center in Wisconsin called Lighthouse, costing $15B+ and set to open in 2028, as part of Stargate",401,https://www.reuters.com/business/openai-oracle-announce-stargate-data-center-site-wisconsin-2025-10-22/


In [18]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

07:56:57 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Processing 116 AI articles for summarization
07:56:57 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Successfully retrieved prompt 'newsagent/extract_summaries' from Langfuse
07:56:57 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Parsed prompt 'newsagent/extract_summaries': model=gpt-5-mini, reasoning_effort=medium, system_len=1273, user_len=43
07:56:57 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Using model 'gpt-5-mini' for summarization
07:56:57 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Starting summarization for 116 articles
07:56:57 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Processing 116 chunks with concurrency 1000


▶ Starting Step 4: extract_summaries


08:16:10 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Extracting metadata from HTML files for 116 articles
08:16:13 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Successfully retrieved prompt 'newsagent/item_distiller' from Langfuse
08:16:13 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Parsed prompt 'newsagent/item_distiller': model=gpt-5-mini, reasoning_effort=medium, system_len=1636, user_len=191
08:16:13 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Processing 116 chunks with concurrency 1000


⏱️  Total execution time: 1200.77s
📊 Final result:
✅ Step 4 extract_summaries completed successfully! Generated AI-powered summaries for 116/116 articles.
💾 Summaries stored in headline DataFrame.


In [19]:
# if we get a refusal, examine why and delete bad rows
bad_stuff =["Anthropic's Jack Cla",
           ]
print([len(s) for s in bad_stuff])
headline_df = state.headline_df
headline_df.loc[(headline_df["title"].str[:20].isin(bad_stuff))]



[20]


,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation,summary,description,tags,short_summary


In [20]:
# remove bad stuff
state.headline_df_to_dict(headline_df.loc[~headline_df["title"].str[:20].isin(bad_stuff)])


In [21]:
# User prompt to run workflow
# user_prompt = "Run step 5, Rate articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("rate_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:16:57 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Rating 116 AI articles using fn_rate_articles
08:16:57 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Calculating article ratings for 116 articles
08:16:57 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Rating recency
08:16:58 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Rating spam probability
08:16:58 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Successfully retrieved prompt 'newsagent/rate_quality' from Langfuse
08:16:58 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, reasoning_effort=medium, system_len=1849, user_len=246


▶ Starting Step 5: rate_articles


08:17:06 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | low quality articles: {0.0: 100, 1.0: 5, 5.384940217754036e-13: 1, 0.0001233946226544279: 1, 0.0028009272492822593: 1, 0.9999909853565219: 1, 0.060086648833494447: 1, 0.9995694234965996: 1, 5.109089028063325e-12: 1, 0.999999091165777: 1, 5.732239023403102e-13: 1, 0.9947799085613173: 1}
08:17:06 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Successfully retrieved prompt 'newsagent/rate_on_topic' from Langfuse
08:17:06 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Parsed prompt 'newsagent/rate_on_topic': model=gpt-4.1, reasoning_effort=medium, system_len=1790, user_len=240
08:17:11 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | on topic articles: {1.0: 99, 0.9999545100305701: 2, 0.7772998664755185: 2, 0.999999448776502: 1, 0.8519528238199198: 1, 0.999999091165777: 1, 0.9959299041310293: 1, 0.9998765647550606: 1, 1.026187963170189e-10: 1, 0.999996587894339: 1, 0.97

,id,site_name,title,bradley_terry,bt_z
75,75,Benzinga,"CoreWeave Is 'Frightening… It's Unbelievable,' Says Expert, Expecting Its Revenue To Multiply Nearly 5x By 2028",26.092259,1.597368
0,0,Financial Times,The AI Shift: where are all the job losses?,24.091305,1.474870
49,49,VentureBeat,Collage style AI artwork of Microsoft CEO Satya Nadella. Credit: VentureBeat made with Midjourney,24.091305,1.474870
58,58,The Guardian,"OpenAI relaxed ChatGPT guardrails just before teen killed himself, family alleges",24.091305,1.474870
30,30,Reuters,"Alibaba launches a free consumer-focused AI chatbot in its Quark app, powered by Qwen 3, and plans to open pre-sales for its $660 Quark AI Glasses on October 24",24.091305,1.474870
...,...,...,...,...,...
89,89,Axios,"UnifyApps, which lets companies build AI chatbots by connecting SaaS apps and data, raised a $50M Series B led by WestBridge, after a $20M Series A in 2024",-24.091305,-1.474870
68,68,Globe Newswire,SCHMID Group secures Major Orders for AI Server PCB Production Equipment,-24.091305,-1.474870
17,17,Hacker Noon,The AI Gatekeeper Test: Why Your Expertise Is Worthless If Algorithms Can't Find It,-24.091305,-1.474870
39,39,Bloomberg,"Why All the Buzz About AGI? And What Is It Anyway?AI companies are in a race to create artificial general intelligence, seeing it as the catalyst for an “intelligence explosion” that reshapes the economy and society. If only they could agree on what AGI would actually look like, and how it might be achieved",-24.091305,-1.474870


08:17:35 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Top 10 ids: [75, 0, 49, 58, 30, 22, 83, 12, 84, 92]
08:17:35 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | After round 1/23: 
08:17:35 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Number of ranking changes: 115
08:17:35 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Sum of absolute ranking changes: 4495.0 (avg rank chg 39.09)
08:17:35 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | ---------------------------------------------------
08:17:35 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Running round 2 of max 23
08:17:35 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | ---------------------------------------------------
08:17:35 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Generated 20 battle batches
08:17:35 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Total stories in batch

...

,id,site_name,title,bradley_terry,bt_z
38,38,Fortune,"Cybersecurity experts warn OpenAI’s ChatGPT Atlas is vulnerable to attacks that could turn it against a user—revealing sensitive data, downloading malware, or worse",13.732032,2.264055
92,92,International Business Times,Is Google Investing Billions in Anthropic? Move Considered a Bold Response to Openai's Atlas,13.732025,2.264054
58,58,The Guardian,"OpenAI relaxed ChatGPT guardrails just before teen killed himself, family alleges",12.122937,1.998757
96,96,DW Innovation,OpenAI launches browser to rival Google Chrome,11.526871,1.900481
27,27,Gizmodo,"OpenAI Weakened ChatGPT’s Self-Harm Guardrails in Lead-Up to Teen’s Death, Lawsuit Says",11.526792,1.900468
...,...,...,...,...,...
42,42,Hacker Noon,Engineering Instagram Captions: A Structured Prompt for AI Tools,-9.281783,-1.530325
4,4,The Wall Street Journal,"AI creative studio Wonder Studios, backed by OpenAI and Google DeepMind execs, raised a $12M seed led by Atomico and aims to release its first content in 2026 (Najat Kantouar/Wall Street Journal)",-9.446017,-1.557403
51,51,The Wall Street Journal,"Top AI researchers and executives in Silicon Valley are working 80 to 100 hours per week, driven by competition, as AI's progress compresses product timelines",-9.769561,-1.610747
109,109,CNN,App that translates speech to sign language in real-time wins top innovation prize,-11.339083,-1.869520


08:19:24 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Top 10 ids: [38, 92, 58, 96, 27, 88, 84, 83, 108, 69]
08:19:24 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | After round 13/23: 
08:19:24 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Number of ranking changes: 48
08:19:24 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Sum of absolute ranking changes: 2236.0 (avg rank chg 19.44)
08:19:24 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | last_two: 17.81, prev_two: 18.59
08:19:24 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | ---------------------------------------------------
08:19:24 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Running round 14 of max 23
08:19:24 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | ---------------------------------------------------
08:19:24 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Generated 23 b

,id,site_name,title,bradley_terry,bt_z
92,92,International Business Times,Is Google Investing Billions in Anthropic? Move Considered a Bold Response to Openai's Atlas,13.536236,2.316871
38,38,Fortune,"Cybersecurity experts warn OpenAI’s ChatGPT Atlas is vulnerable to attacks that could turn it against a user—revealing sensitive data, downloading malware, or worse",13.536235,2.316871
58,58,The Guardian,"OpenAI relaxed ChatGPT guardrails just before teen killed himself, family alleges",11.927112,2.041452
96,96,DW Innovation,OpenAI launches browser to rival Google Chrome,11.331124,1.939443
27,27,Gizmodo,"OpenAI Weakened ChatGPT’s Self-Harm Guardrails in Lead-Up to Teen’s Death, Lawsuit Says",11.331022,1.939425
...,...,...,...,...,...
42,42,Hacker Noon,Engineering Instagram Captions: A Structured Prompt for AI Tools,-8.816630,-1.509060
4,4,The Wall Street Journal,"AI creative studio Wonder Studios, backed by OpenAI and Google DeepMind execs, raised a $12M seed led by Atomico and aims to release its first content in 2026 (Najat Kantouar/Wall Street Journal)",-8.958686,-1.533375
51,51,The Wall Street Journal,"Top AI researchers and executives in Silicon Valley are working 80 to 100 hours per week, driven by competition, as AI's progress compresses product timelines",-9.084268,-1.554869
109,109,CNN,App that translates speech to sign language in real-time wins top innovation prize,-10.845130,-1.856260


08:19:33 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Top 10 ids: [92, 38, 58, 96, 27, 88, 84, 83, 108, 69]
08:19:33 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | After round 14/23: 
08:19:33 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Number of ranking changes: 44
08:19:33 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Sum of absolute ranking changes: 1680.0 (avg rank chg 14.61)
08:19:33 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | last_two: 17.03, prev_two: 15.82
08:19:33 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Increase in avg rank change, stopping
08:19:33 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Low rated articles: 1
08:19:33 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | low rated article: Watch Your Wallet: ChatGPT Atlas Might Try to Buy the Wrong Product for You -3.31776067419408
08:19:35 | NewsletterAgent.test_newsletter_20

⏱️  Total execution time: 157.16s
📊 Final result:
✅ Step 5 rate_articles completed successfully! Rated 114 articles with average rating 6.4/10.
⭐ High quality articles (≥7.0): 51
💾 Ratings stored in persistent state.


In [22]:
headline_df=state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df.sort_values("rating", ascending=False)[['site_name', 'title', 'rating', 'short_summary']])


,site_name,title,rating,short_summary
0,Financial Times,The AI Shift: where are all the job losses?,12.466272,"Payroll and job-posting data show AI-linked displacement in freelancers and junior coders: Stanford researchers estimate young software-developer employment is nearly 20% down since late 2022, and a Swedish union reports AI implementation rose from 29% to about 66%."
27,Gizmodo,"OpenAI Weakened ChatGPT’s Self-Harm Guardrails in Lead-Up to Teen’s Death, Lawsuit Says",10.557499,"An amended lawsuit alleges OpenAI's May 8, 2024 and Feb 12, 2025 model-spec changes weakened ChatGPT's self-harm guardrails, shifting refusals to engagement as Adam Raine's chats rose from dozens daily to over 300 per day and he died April 11, 2025."
38,Fortune,"Cybersecurity experts warn OpenAI’s ChatGPT Atlas is vulnerable to attacks that could turn it against a user—revealing sensitive data, downloading malware, or worse",10.545731,"OpenAI launched ChatGPT Atlas, an AI browser with ""browser memories"" and an experimental agent mode that cybersecurity experts warn creates prompt-injection attack vectors capable of exposing emails, passwords, clipboard contents, downloading malware or performing unauthorized transactions."
2,Financial Times,AI’s rapid evolution demands more flexible training,10.268962,"Businesses must renew modular AI training frequently, as regular AI training rose from 16% in 2024 to 28% in 2025, and firms pair rapid upskilling with governance and responsible-AI programmes to manage risk and capture value."
12,The Verge,Amazon claims the headline isn’t robots taking jobs as it reveals new cost-cutting robots,10.232275,"Amazon revealed ten robots and delivery technologies, including Blue Jay, which handles about 75% of item types and consolidates three robotic stations, and said it will hire 250,000 seasonal workers while CEO Andy Jassy warned AI will reduce some jobs."
...,...,...,...,...
112,Biztoc,What Is One of the Best Artificial Intelligence (AI) Stocks to Buy Now?,2.189466,"Microsoft is identified as one of the best AI stocks to buy now, with AI expected to broadly embed across its software portfolio, boosting offerings and supporting continued revenue growth without redefining its core software business."
113,Channel News Asia,India proposes strict IT rules for labelling deepfakes amid AI misuse,2.065509,India proposes IT rules requiring labeling of deepfakes to address AI misuse.
111,Biztoc,Better Artificial Intelligence Stock: BigBear.ai vs. C3.ai,1.716957,"BigBear.ai and C3.ai reported year-over-year sales declines in 2025, with BigBear.ai hit by federal budget cuts and C3.ai's sales execution deteriorating after its CEO stepped down."
110,Social Media Explorer,The Human Touch Behind AI: How Changescape Web Balances Automation with Authenticity,1.419355,"Changescape Web, led by founder Ken Tucker, uses AI-driven tools including the AI Assistant IQ virtual phone/chat assistant while human strategists train and review outputs to capture leads, provide 24/7 service, optimize SEO, and improve clients' visibility and response times."


In [23]:
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    display(Markdown(f"""
{row.rating:.1f} - [{row.title}]({row.url}) - {row.site_name}

{row.short_summary}
    """))
    


12.5 - [The AI Shift: where are all the job losses?](https://www.ft.com/content/3d2669e3-c05e-48c9-8bb3-893c1d66de2e) - Financial Times

Payroll and job-posting data show AI-linked displacement in freelancers and junior coders: Stanford researchers estimate young software-developer employment is nearly 20% down since late 2022, and a Swedish union reports AI implementation rose from 29% to about 66%.
    


10.6 - [OpenAI Weakened ChatGPT’s Self-Harm Guardrails in Lead-Up to Teen’s Death, Lawsuit Says](https://gizmodo.com/openai-weakened-chatgpts-self-harm-guardrails-in-lead-up-to-teens-death-lawsuit-says-2000675800) - Gizmodo

An amended lawsuit alleges OpenAI's May 8, 2024 and Feb 12, 2025 model-spec changes weakened ChatGPT's self-harm guardrails, shifting refusals to engagement as Adam Raine's chats rose from dozens daily to over 300 per day and he died April 11, 2025.
    


10.5 - [Cybersecurity experts warn OpenAI’s ChatGPT Atlas is vulnerable to attacks that could turn it against a user—revealing sensitive data, downloading malware, or worse](https://fortune.com/2025/10/23/cybersecurity-vulnerabilities-openai-chatgpt-atlas-ai-browser-leak-user-data-malware-prompt-injection/) - Fortune

OpenAI launched ChatGPT Atlas, an AI browser with "browser memories" and an experimental agent mode that cybersecurity experts warn creates prompt-injection attack vectors capable of exposing emails, passwords, clipboard contents, downloading malware or performing unauthorized transactions.
    


10.3 - [AI’s rapid evolution demands more flexible training](https://www.ft.com/content/177dab62-efc7-4485-9cf2-c78e94ac0302) - Financial Times

Businesses must renew modular AI training frequently, as regular AI training rose from 16% in 2024 to 28% in 2025, and firms pair rapid upskilling with governance and responsible-AI programmes to manage risk and capture value.
    


10.2 - [Amazon claims the headline isn’t robots taking jobs as it reveals new cost-cutting robots](https://www.theverge.com/news/805098/amazon-robots-ai-warehouses) - The Verge

Amazon revealed ten robots and delivery technologies, including Blue Jay, which handles about 75% of item types and consolidates three robotic stations, and said it will hire 250,000 seasonal workers while CEO Andy Jassy warned AI will reduce some jobs.
    

...


2.2 - [What Is One of the Best Artificial Intelligence (AI) Stocks to Buy Now?](https://biztoc.com/x/0290144423dc727f) - Biztoc

Microsoft is identified as one of the best AI stocks to buy now, with AI expected to broadly embed across its software portfolio, boosting offerings and supporting continued revenue growth without redefining its core software business.
    


2.1 - [India proposes strict IT rules for labelling deepfakes amid AI misuse](https://www.channelnewsasia.com/business/india-proposes-strict-it-rules-labelling-deepfakes-amid-ai-misuse-5417516) - Channel News Asia

India proposes IT rules requiring labeling of deepfakes to address AI misuse.
    


1.7 - [Better Artificial Intelligence Stock: BigBear.ai vs. C3.ai](https://biztoc.com/x/348d8f965bc343c7) - Biztoc

BigBear.ai and C3.ai reported year-over-year sales declines in 2025, with BigBear.ai hit by federal budget cuts and C3.ai's sales execution deteriorating after its CEO stepped down.
    


1.4 - [The Human Touch Behind AI: How Changescape Web Balances Automation with Authenticity](https://socialmediaexplorer.com/business-innovation-2/the-human-touch-behind-ai-how-changescape-web-balances-automation-with-authenticity/) - Social Media Explorer

Changescape Web, led by founder Ken Tucker, uses AI-driven tools including the AI Assistant IQ virtual phone/chat assistant while human strategists train and review outputs to capture leads, provide 24/7 service, optimize SEO, and improve clients' visibility and response times.
    


1.4 - [App that translates speech to sign language in real-time wins top innovation prize](https://www.cnn.com/world/africa/speech-to-sign-language-app-wins-africa-innovation-prize-spc) - CNN

An app that translates speech to sign language in real time won a top innovation prize.
    

In [24]:
# User prompt to run workflow
# user_prompt = "Run step 6, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:19:35 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Starting topic extraction for clustering
08:19:35 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Successfully retrieved prompt 'newsagent/extract_topics' from Langfuse
08:19:35 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Parsed prompt 'newsagent/extract_topics': model=gpt-5-nano, reasoning_effort=medium, system_len=1100, user_len=80
08:19:35 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Using model 'gpt-5-nano' for topic extraction
08:19:35 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Processing 114 articles for topic extraction
08:19:35 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Processing 12 chunks with concurrency 16


▶ Starting Step 6: cluster_topics


08:20:17 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Successfully extracted 421 total topics across articles
08:20:17 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Starting canonical topic classification for 101 topics
08:20:17 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
08:20:17 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Parsed prompt 'newsagent/canonical_topic': model=gpt-5-nano, reasoning_effort=medium, system_len=426, user_len=179
08:20:17 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
08:20:17 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Parsed prompt 'newsagent/canonical_topic': model=gpt-5-nano, reasoning_effort=medium, system_len=426, user_len=179
08:20:17 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Succ

Starting optimization with 200 trials...
Original embedding shape: (114, 3072)


  0%|          | 0/200 [00:00<?, ?it/s]

...

=== HDBSCAN Parameters ===
min_cluster_size:   2
min_samples:        2
n_components:       49
=== Clustering Quality Metrics ===
Number of clusters: 13
Noise points: 54 (47.4%)
Average cluster size: 4.6 ± 3.2
Cluster size range: 2 - 11
=== Quality Scores ===
Silhouette Score: 0.264 (higher is better)
Calinski-Harabasz Score: 6.3 (higher is better)
Davies-Bouldin Score: 1.270 (lower is better)
HDBSCAN Validity Index: 0.126
Composite Score: 0.195 (higher is better)

[I 2025-10-23 08:23:06,026] Trial 198 finished with value: -0.1949692356613259 and parameters: {'n_components': 49, 'min_cluster_size': 2, 'min_samples': 2}. Best is trial 69 with value: -0.2560648305251862.


/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
08:23:06 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Successfully retrieved prompt 'newsagent/topic_writer' from Langfuse
08:23:06 | NewsletterA

=== HDBSCAN Parameters ===
min_cluster_size:   2
min_samples:        2
n_components:       20
=== Clustering Quality Metrics ===
Number of clusters: 12
Noise points: 52 (45.6%)
Average cluster size: 5.2 ± 3.0
Cluster size range: 2 - 11
=== Quality Scores ===
Silhouette Score: 0.322 (higher is better)
Calinski-Harabasz Score: 11.9 (higher is better)
Davies-Bouldin Score: 1.129 (lower is better)
HDBSCAN Validity Index: 0.172
Composite Score: 0.247 (higher is better)

[I 2025-10-23 08:23:06,049] Trial 199 finished with value: -0.24670055680252084 and parameters: {'n_components': 20, 'min_cluster_size': 2, 'min_samples': 2}. Best is trial 69 with value: -0.2560648305251862.

Optimization completed!
Best composite score: 0.2561
Best parameters: {'n_components': 33, 'min_cluster_size': 3, 'min_samples': 2}

=== Results with Best Parameters ===
Reduced dimensions from 3072 to 33
=== Clustering Quality Metrics ===
Number of clusters: 7
Noise points: 50 (43.9%)
Average cluster size: 9.1 ± 8.6
C

08:23:15 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | 4: AI, Work, and Governance
08:23:15 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | The AI Shift: where are all the job losses? (Labor Market, Jobs And Careers, Outsourcing, AI Adoption, Gen AI, Job Automation, Freelancers)
Jobseekers of the future: approach AI with scepticism and dexterity (Labor Market, Upskilling, AI Literacy, Skilled Trades, Jobs And Careers, Gen AI, Economics)
AI’s rapid evolution demands more flexible training (AI Training, Upskilling, Governance, Education, Job Automation, Responsible AI, Economics)
AI tutors show their potential in interactive workplace learning (Workplace Learning, Human Oversight, Safety And Alignment, Ethics, Compliance Training, Education, Jobs And Careers)
An Architect's View: Why AI Needs an Ethical OS, Not More Rules (Safety And Alignment, Governance, Policy And Regulation, Ethics, Autonomous Vehicles, Opinion, Artificial-intelligence)
From Automati

⏱️  Total execution time: 258.61s
📊 Final result:
✅ Step 6 cluster_topics completed successfully! Organized 114 articles into topic clusters.


In [25]:
# User prompt to run workflow
# user_prompt = "Run step 7, select section topics"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("select_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:23:53 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Category proposal
08:23:53 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Successfully retrieved prompt 'newsagent/cat_proposal' from Langfuse
08:23:53 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Parsed prompt 'newsagent/cat_proposal': model=gpt-5-mini, reasoning_effort=medium, system_len=638, user_len=1179


▶ Starting Step 7: select_sections


08:24:54 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Cleaning up initial categories: ['AI Infrastructure and GPU Competition', 'AI Job Displacement Patterns', 'Election Deepfakes and Disinformation', 'ChatGPT Atlas Browser', 'AI Browser Security Risks', 'Superintelligence Ban Petition', 'Copilot For Exchange On Prem', 'ChatGPT Erotic Content Policy', 'PyTorch Monarch Framework', 'Microsoft Azure Strategic Push', 'Alibaba Quark AI Glasses', 'OpenAI Launches ChatGPT Atlas Browser', 'Amazon Warehouse Automation', 'Other', 'AI Cloud Competition Shift', 'AI, Work, and Governance', 'Weakened ChatGPT Guardrails', 'Hidden Enterprise Data Problem', 'SRE To Autonomous AIOps', "Alibaba's Qwen AI Expansion", 'Airbnb Picks Alibaba Qwen', 'Calls to Ban AI Superintelligence', 'IBM GPU Rental Strategy', 'Production Ready RAG Pipelines', 'Anthropic Google Cloud Talks', 'AI Driven DDoS Attacks', 'Customer Controlled AI Architectures', 'Deepfakes Threaten Elections', 'Agentic Coding T

⏱️  Total execution time: 143.81s
📊 Final result:
Categories and article counts:
cat
AI Infrastructure GPU Competition    10
AI Job Displacement                   8
AI Startup Funding Rounds             7
AI Work Governance                   24
Agentic Coding Tools                  3
Alibaba Qwen Expansion                4
Amazon Warehouse Automation           3
Election Deepfakes                    4
Microsoft Azure Expansion             2
OpenAI ChatGPT Atlas                  5
Other                                19
Production RAG Pipelines              7
Superintelligence Ban Calls           2


In [26]:
# User prompt to run workflow
# user_prompt = "Run step 8, draft sections"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("draft_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:26:17 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Step 8a: Selecting stories from 98 total
08:26:17 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | 29 must-include stories with rating > 8
08:26:17 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | 69 stories with rating <= 8


▶ Starting Step 8: draft_sections


08:26:21 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Selected 69 diverse Tier 2 stories via MMR
08:26:21 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Total selected stories: 98 (target: 100)
08:26:21 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Drafting sections for 13 categories (including Other) from selected stories
08:26:21 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Successfully retrieved prompt 'newsagent/write_section' from Langfuse
08:26:21 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Parsed prompt 'newsagent/write_section': model=gpt-5-mini, reasoning_effort=high, system_len=1854, user_len=22
08:31:38 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Drafted 13 sections
08:31:38 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Reassigned 5 pruned stories to Other section
08:31:38 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Created ne

⏱️  Total execution time: 673.15s
📊 Final result:
✅ Step 8 draft_sections completed successfully! Drafted 61 stories across 13 sections


In [27]:
newsletter_section_df = state.newsletter_section_df

newsletter_section_df

,cat,section_title,id,headline,rating,prune,links,sort_order
0,AI Job Displacement,"Bots, Jobs, and Paychecks",0,AI displaces freelancers and junior developers...,12.466272,False,[Financial Times](https://www.ft.com/content/3...,2
1,AI Job Displacement,"Bots, Jobs, and Paychecks",66,PwC: AI-skilled workers earn a 56% wage premiu...,5.330195,False,[Forbes](https://www.forbes.com/sites/iese/202...,2
2,AI Job Displacement,"Bots, Jobs, and Paychecks",1,Study: Only 5% of US college students fully pa...,10.142679,False,[Financial Times](https://www.ft.com/content/5...,2
3,AI Job Displacement,"Bots, Jobs, and Paychecks",70,Meta to cut about 600 AI-division staff despit...,7.050327,False,[Mashable](https://mashable.com/article/meta-l...,2
4,AI Job Displacement,"Bots, Jobs, and Paychecks",23,AI automates hiring: bots apply to bots as res...,7.575548,False,[Hacker Noon](https://hackernoon.com/when-ai-c...,2
...,...,...,...,...,...,...,...,...
56,Other,"AI: Boom, Bots, and Business",14,Apple Intelligence beta brings six everyday AI...,7.530980,False,[CNET](https://www.cnet.com/tech/services-and-...,12
57,AI Startup Funding Rounds,AI Funding & Momentum,63,Ulangizi AI uses LLMs and local guidelines to ...,5.476711,False,[Rest of World](https://restofworld.org/2025/m...,5
58,AI Work Governance,"AI: Rules, Risks, and Control",21,"Brands (Heineken, Aerie, Polaroid) run anti-AI...",6.184299,False,[Business Insider](https://www.businessinsider...,0
59,Microsoft Azure Expansion,"Azure, AI and the Bottom Line",49,"Nadella: Microsoft mobilizes 34,000 engineers,...",8.802542,False,[VentureBeat](https://venturebeat.com/ai/what-...,9


In [28]:
Markdown(state.display_newsletter().replace("$","\\\$"))


## AI: Rules, Risks, and Control
- Lawsuit alleges OpenAI weakened ChatGPT self-harm guardrails before teen's death - [Gizmodo](https://gizmodo.com/openai-weakened-chatgpts-self-harm-guardrails-in-lead-up-to-teens-death-lawsuit-says-2000675800)
- 78% of large firms avoid sending proprietary data to third‑party AI vendors, IDC says - [Hacker Noon](https://hackernoon.com/your-data-your-rules-ais-demand-for-customer-controlled-architectures)
- Workers say xAI used short Hollywood clips, including Hellboy II, to annotate Grok training data - [Business Insider](https://www.businessinsider.com/grok-workers-hellboy-hollywood-movies-elon-musk-xai-2025-10)
- UK funds 15 AI sandbox projects with £8.9m to test governance and use cases - [The Register](https://www.theregister.com/2025/10/23/uk_ai_regulations/)
- OpenAI will let verified adults generate erotic ChatGPT content starting December - [Wired](https://www.wired.com/story/chatgpt-horny-era/)
- India proposes IT rules requiring platforms to detect, tag and label deepfakes - [Yahoo](https://www.yahoo.com/news/articles/india-proposes-rules-tackle-deepfakes-084635858.html)
- Brands (Heineken, Aerie, Polaroid) run anti-AI marketing as Pew finds 50% more Americans worried than excited; 57% view AI risks high - [Business Insider](https://www.businessinsider.com/brands-reject-ai-aerie-heineken-polaroid-marketing-2025-10)
## AI Compute Arms Race
- Google negotiating multibillion‑to‑tens‑of‑billions TPU supply deal with Anthropic - [International Business Times](https://www.ibtimes.co.uk/google-investing-billions-anthropic-move-considered-bold-response-openais-atlas-1749402)
- Bernstein: AWS trailing AI cloud race due to GPU capacity constraints - [Business Insider](https://www.businessinsider.com/amazon-last-ai-cloud-race-why-not-death-sentence-2025-10)
- Musk says Tesla AI5 chips, built by Samsung and TSMC, will target compute equal to ~81,000 H100s - [CNBC](https://www.cnbc.com/2025/10/22/elon-musk-tesla-ai5-nvidia.html)
- PyTorch Monarch scales to 2,048 GPUs, adds sharded tensors, cuts recovery overheads by ≥60% - [PyTorch](https://pytorch.org/blog/introducing-pytorch-monarch/)
- Analyst projects CoreWeave revenue from \\$16M (2022) to \\$5.3B (2025) and \\$25B (2028) - [Benzinga](https://www.benzinga.com/markets/equities/25/10/48373546/coreweave-is-frightening-its-unbelievable-says-expert-expecting-its-revenue-to-multiply-nearly-5x-by-2028)
- Nvidia data‑center revenue rises 56% year‑over‑year to \\$41.1B - [The Motley Fool](https://www.fool.com/investing/2025/10/21/think-you-missed-the-boat-on-nvidia-heres-the-no-1/)
- OpenAI, Oracle and Vantage to build \\$15B+ 'Stargate' data center in Wisconsin by 2028 - [Reuters](https://www.reuters.com/business/openai-oracle-announce-stargate-data-center-site-wisconsin-2025-10-22/)
- SCHMID Group wins major orders to supply PCB production equipment for AI server boards, including a full wet-process line - [Globe Newswire](https://www.globenewswire.com/news-release/2025/10/22/3170998/0/en/SCHMID-Group-secures-Major-Orders-for-AI-Server-PCB-Production-Equipment.html)
- Nokia shares surge as AI and cloud demand lift profit, biggest jump since 2021 - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-23/nokia-profit-surges-past-estimates-on-network-growth)
## Bots, Jobs, and Paychecks
- AI displaces freelancers and junior developers; young developers' employment down ~20% since late 2022 - [Financial Times](https://www.ft.com/content/3d2669e3-c05e-48c9-8bb3-893c1d66de2e)
- Study: Only 5% of US college students fully parse Bleak House opening; experts call for AI literacy and continuous upskilling - [Financial Times](https://www.ft.com/content/5e78cd00-ca0e-488d-87d0-8fef8a84f688)
- AI workplace learning moves to micro-lessons and VR simulations; Bank of America's role-play used 1.8M times, study finds AI coaches match human trainers - [Financial Times](https://www.ft.com/content/1f915aa9-c61c-4f55-b27e-47d0a0f2c016)
- AI automates hiring: bots apply to bots as resume screening accelerates; calls grow for recruiting standards - [Hacker Noon](https://hackernoon.com/when-ai-candidates-apply-to-ai-recruiters-the-end-of-human-hiring)
- Meta to cut about 600 AI-division staff despite continued AI investments - [Mashable](https://mashable.com/article/meta-lay-off-600-ai)
- PwC: AI-skilled workers earn a 56% wage premium; firms urged to redesign jobs for GenAI - [Forbes](https://www.forbes.com/sites/iese/2025/10/22/hack-yourself-how-to-find-uses-for-ai-in-everything-you-do-at-work/)
## ChatGPT Atlas: Browser Breakthrough
- Experts warn ChatGPT Atlas Agent Mode can leak emails, passwords and clipboard via prompt injection - [Fortune](https://fortune.com/2025/10/23/cybersecurity-vulnerabilities-openai-chatgpt-atlas-ai-browser-leak-user-data-malware-prompt-injection/)
- Ars Technica publishes 10-point hands-on review of ChatGPT Atlas Agent Mode - [Ars Technica](https://arstechnica.com/features/2025/10/we-let-openais-agent-mode-surf-the-web-for-us-heres-what-happened/)
- OpenAI to add profiles, tab groups, ad blocker, 1Password fixes and Drive/Excel integration to ChatGPT Atlas - [The Verge](https://www.theverge.com/news/805160/openai-chatgpt-atlas-updates)
- OpenAI launches ChatGPT Atlas to challenge Google Search, pursue monetization and India expansion - [Live Mint](https://www.livemint.com/companies/news/openai-chatgpt-atlas-ai-browser-google-search-india-google-silicon-valley-internet-ai-platform-big-tech-data-startup-11761118063947.html)
- CNET recommends DALL·E 3 in ChatGPT Plus, warns of legal risks from training data - [CNET](https://www.cnet.com/tech/services-and-software/create-the-perfect-ai-image-image-generator-tips-for-openai-google-and-canva/)
- APK teardown finds hidden ChatGPT voice mode; OpenAI moving voice into chat with live transcripts - [Android Authority](https://www.androidauthority.com/chatgpt-voice-mode-chat-integration-apk-teardown-3609526/)
## Production AI Playbook
- Poor data strategy, not compute, is derailing enterprise visual AI; curated tooling cut review workload 80% - [Fast Company](https://www.fastcompany.com/91423778/the-hidden-data-problem-killing-enterprise-ai-projects-implementing-ai)
- Production-grade RAG: hybrid retrieval, caching, verification and routing cut latency, hallucinations, and costs - [Hacker Noon](https://hackernoon.com/10-23-2025-techbeat)
- Audit of 23 AI startups finds most are API wrappers; 91% lack vector DBs and none fine-tune models - [Hacker Noon](https://hackernoon.com/i-reverse-engineered-how-23-ai-first-companies-actually-build-their-products-and-the-tech-stack-is)
- Colab-ready enterprise assistant blueprint: FAISS retrieval, MiniLM, PII redaction, RBAC and 180-day audit logs - [MarkTechPost](https://www.marktechpost.com/2025/10/22/how-to-design-a-fully-functional-enterprise-ai-assistant-with-retrieval-augmentation-and-policy-guardrails-using-open-source-ai-models/)
## AI Funding & Momentum
- UnifyApps raises \\$50M Series B at ~\\$250M valuation, appoints Ragy Thomas chairman and co‑CEO - [Yahoo](https://finance.yahoo.com/news/ai-startup-unifyapps-raises-50-113233926.html)
- Wonder Studios, backed by OpenAI and DeepMind, raises \\$12M seed to pursue AI-generated content in 2026 - [The Wall Street Journal](https://www.wsj.com/tech/ai/openai-backed-creative-studio-raises-12-million-as-it-eyes-ai-content-drive-cad81e1f)
- Investors pivot on Tesla, favor AI-focused software, autonomy and robotics spending - [Financial Post](https://financialpost.com/commodities/energy/electric-vehicles/tesla-stops-growing-investors-pivot-to-ai)
- Ulangizi AI uses LLMs and local guidelines to provide Chichewa crop diagnostics via WhatsApp to Malawian farmers - [Rest of World](https://restofworld.org/2025/malawi-ulangizi-ai-farming-chatbot/)
- Healthcare AI Hyro raises \\$45M, says it reaches 30M patients across 45 U.S. health systems - [Tech in Asia](https://www.techinasia.com/news/us-healthcare-ai-startup-hyro-bags-45m)
- China's robovan maker Neolix raises \\$600M for R&D, eyes 2026 Hong Kong IPO - [Nikkei Asia](https://asia.nikkei.com/business/technology/chinese-robovan-startup-neolix-raises-600m-for-tech-product-development)
## Alibaba's Qwen AI Wave
- Alibaba adds Qwen3 chatbot to Quark app; Quark AI Glasses pre-sale starts Oct. 24 - [Reuters](https://www.reuters.com/world/asia-pacific/alibaba-launches-ai-chatbot-service-renewed-consumer-push-2025-10-23/) [South China Morning Post](https://www.scmp.com/tech/tech-trends/article/3330034/alibabas-quark-app-adds-ai-chatbot-challenge-bytedances-doubao-tencents-yuanbao)
- Airbnb adopts Alibaba's Qwen for customer service, citing faster, cheaper support - [Yahoo](https://finance.yahoo.com/news/airbnb-picks-alibabas-qwen-over-093000045.html)
- Alibaba debuts two visual-language models for Alibaba Cloud's multimodal AI - [Biztoc](https://biztoc.com/x/fc79d5e56ee57ccb)
## Faking It: Deepfakes Rising
- Deepfakes mimic RTÉ presenters and politicians ahead of Irish presidential vote - [The Journal](https://www.thejournal.ie/factcheck-ai-generated-videos-and-images-irish-presidential-race-atas-2025-catherine-connolly-heather-humphreys-6850517-Oct2025/)
- Sam Altman warns of AI dangers after Sora 2 sparks viral deepfakes - [Investopedia](https://www.investopedia.com/i-expect-some-really-bad-stuff-to-happen-says-the-ceo-of-chatgpt-s-parent-company-heres-what-hes-talking-about-11833105)
- Maryland woman charged after sharing AI-generated 'intruder' photo and filing a false report - [The Washington Post](https://www.washingtonpost.com/dc-md-va/2025/10/23/ai-fake-intruder/)
## AI Coding Tools Unleashed
- Autonomous AIOps shifts SRE practices — ~40% adoption by 2024; bank auto-resolved 62% of incidents - [Hacker Noon](https://hackernoon.com/from-automation-to-autonomy-how-ai-is-transforming-site-reliability-engineering)
- Sourcegraph opens Amp to all with VS Code extension and CLI; reportedly generates 70–80% of commits - [Hacker Noon](https://hackernoon.com/sourcegraph-drops-the-waitlist-amp-its-agentic-coding-tool-opens-to-everyone)
- Sonar profiles LLM 'coding personalities' and warns they produce insecure, debt‑prone code - [Forbes](https://www.forbes.com/sites/lanceeliot/2025/10/22/the-best-way-to-vibe-code-is-to-first-know-the-ai-coding-personality-that-you-are-dealing-with/)
- Gemini CLI v0.9 adds PTY and interactive shell; now available via npm - [Google Developers Blog](https://developers.googleblog.com/en/say-hello-to-a-new-level-of-interactivity-in-gemini-cli/)
## Azure, AI and the Bottom Line
- Microsoft surveys customers on Copilot for on‑prem Exchange, probing data-sending and safeguard needs - [The Register](https://www.theregister.com/2025/10/23/copilot_exchange_server/)
- Nadella: Microsoft mobilizes 34,000 engineers, 400+ Azure datacenters, 2 GW new Azure AI compute and 11,000+ AI models - [VentureBeat](https://venturebeat.com/ai/what-enterprises-can-take-away-from-microsoft-ceo-satya-nadellas-shareholder)
- Analysts: Microsoft among top AI stocks as AI features boost software revenue - [Biztoc](https://biztoc.com/x/0290144423dc727f)
## Robots on the Move
- Amazon unveils 10 robots including Blue Jay (handles ~75% of item types), warns AI will cut jobs; plans 250,000 seasonal hires - [The Verge](https:\/\/www.theverge.com\/news\/805098\/amazon-robots-ai-warehouses)
- Amazon pilots AR glasses to give delivery drivers hands-free scanning and navigation - [GeekWire](https:\/\/www.geekwire.com\/2025\/amazon-unveils-ai-powered-augmented-reality-glasses-for-delivery-drivers\/)
- FIX 2025 mobility expo in Daegu spotlights EVs, VTOLs, autonomous driving and humanoid robots - [DigiTimes](https:\/\/www.digitimes.com\/news\/a20251022VL209\/south-korea-flying-car-robotics-xpeng-2025.html)
## Pause the AGI Race
- 700+ signatories including Geoffrey Hinton, Richard Branson and Prince Harry call to pause superintelligence pursuit until safety and public consent - [The Economic Times](https://m.economictimes.com/tech/artificial-intelligence/celebrities-ai-giants-urge-end-to-superintelligence-quest/articleshow/124738664.cms)
- OpenAI and Anthropic race toward AGI, warn of possible 'intelligence explosion' and disagree on AGI definition - [Bloomberg](https://www.bloomberg.com/news/features/2025-10-23/what-is-agi-why-openai-anthropic-are-targeting-artificial-general-intelligence)
## AI: Boom, Bots, and Business
- AI workplace learning moves to micro-lessons and VR simulations; Bank of America's role-play used 1.8M times, study finds AI coaches match human trainers - [Financial Times](https://www.ft.com/content/1f915aa9-c61c-4f55-b27e-47d0a0f2c016)
- Researchers say AI enables adaptive, real-time multi-vector DDoS attacks up to 287% more sophisticated - [Hacker Noon](https://hackernoon.com/smart-attacks-smarter-defenses-how-ai-is-transforming-ddos-attacks?source=rss)
- Apple Intelligence beta brings six everyday AI features to iPhone 15 Pro, 16 and 17 models - [CNET](https://www.cnet.com/tech/services-and-software/yes-apple-intelligence-can-be-useful-here-are-6-features-i-use-regularly/)
- McKinsey warns AI could cut banks' profits by as much as \\$170 billion - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-23/mckinsey-says-bank-profits-face-possible-170-billion-ai-hit)
- Musk says Tesla's Optimus could perform surgery; production starts next year with a 1M/year target - [Business Insider](https://www.businessinsider.com/elon-musk-says-optimus-will-be-an-incredible-surgeon-2025-10)


In [29]:
# User prompt to run workflow
# user_prompt = "Run step 9, finalize newsletter"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("finalize_newsletter")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:37:30 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Step 9: write title
08:37:30 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Compiled 61 items into markdown input
08:37:30 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Successfully retrieved prompt 'newsagent/generate_newsletter_title' from Langfuse
08:37:30 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Parsed prompt 'newsagent/generate_newsletter_title': model=gpt-5-mini, reasoning_effort=high, system_len=1082, user_len=225


▶ Starting Step 9: finalize_newsletter


08:37:56 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Generated title: Regulation Tightens as Compute Arms Race Accelerates; AI Displaces Jobs
08:37:56 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Step 9b: Starting critic-optimizer loop
08:37:56 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Successfully retrieved prompt 'newsagent/critique_newsletter' from Langfuse
08:37:56 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Parsed prompt 'newsagent/critique_newsletter': model=gpt-5-mini, reasoning_effort=medium, system_len=2758, user_len=647
08:37:56 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Successfully retrieved prompt 'newsagent/improve_newsletter' from Langfuse
08:37:56 | NewsletterAgent.test_newsletter_20251023074854730516 | INFO | Parsed prompt 'newsagent/improve_newsletter': model=gpt-5-mini, reasoning_effort=high, system_len=1320, user_len=195
08:37:56 | NewsletterAgent.test_newsletter_20

⏱️  Total execution time: 523.48s
📊 Final result:
🎉 Step 9 finalize_newsletter completed successfully!
⭐ Final quality score: 8.0/10
🔄 Completed 2 critic-optimizer iteration(s)
📰 Newsletter stored in persistent state and emailed
✅ Workflow complete! All 9 steps finished successfully.


In [30]:
display(Markdown(state.final_newsletter.replace("$", "\\\$")))

# Regulation Tightens as Compute Arms Race Accelerates; AI Displaces Jobs

## Rulebook & Red Flags — Regulation & Safety
- Lawsuit alleges OpenAI weakened ChatGPT self-harm guardrails before teen's death - [Gizmodo](https://gizmodo.com/openai-weakened-chatgpts-self-harm-guardrails-in-lead-up-to-teens-death-lawsuit-says-2000675800)
- 78% of large firms avoid sending proprietary data to third‑party AI vendors, IDC says - [Hacker Noon](https://hackernoon.com/your-data-your-rules-ais-demand-for-customer-controlled-architectures)
- Workers say xAI used short Hollywood clips, including Hellboy II, to annotate Grok training data - [Business Insider](https://www.businessinsider.com/grok-workers-hellboy-hollywood-movies-elon-musk-xai-2025-10)
- UK funds 15 AI sandbox projects with £8.9m to test governance and use cases - [The Register](https://www.theregister.com/2025/10/23/uk_ai_regulations/)
- OpenAI to allow verified adults to generate erotic ChatGPT content starting December - [Wired](https://www.wired.com/story/chatgpt-horny-era/)
- India proposes IT rules requiring platforms to detect, tag and label deepfakes - [Yahoo](https://www.yahoo.com/news/articles/india-proposes-rules-tackle-deepfakes-084635858.html)
- Brands run anti-AI marketing as Pew finds 50% more Americans worried than excited; 57% view AI risks high - [Business Insider](https://www.businessinsider.com/brands-reject-ai-aerie-heineken-polaroid-marketing-2025-10)

## Press Pause — AGI Safety Call
- 700+ signatories including Geoffrey Hinton, Richard Branson and Prince Harry call to pause superintelligence pursuit until safety and public consent - [The Economic Times](https://m.economictimes.com/tech/artificial-intelligence/celebrities-ai-giants-urge-end-to-superintelligence-quest/articleshow/124738664.cms)
- OpenAI and Anthropic race toward AGI; firms warn of possible 'intelligence explosion' and disagree on AGI definition - [Bloomberg](https://www.bloomberg.com/news/features/2025-10-23/what-is-agi-why-openai-anthropic-are-targeting-artificial-general-intelligence)

## Fakes & Threats — Deepfakes and Attacks
- Deepfakes mimic RTÉ presenters and politicians ahead of Irish presidential vote - [The Journal](https://www.thejournal.ie/factcheck-ai-generated-videos-and-images-irish-presidential-race-atas-2025-catherine-connolly-heather-humphreys-6850517-Oct2025/)
- Sam Altman warns of AI dangers after Sora 2 sparks viral deepfakes - [Investopedia](https://www.investopedia.com/i-expect-some-really-bad-stuff-to-happen-says-the-ceo-of-chatgpt-s-parent-company-heres-what-hes-talking-about-11833105)
- Maryland woman charged after sharing AI-generated 'intruder' photo and filing a false report - [The Washington Post](https://www.washingtonpost.com/dc-md-va/2025/10/23/ai-fake-intruder/)
- Researchers say AI enables adaptive, real-time multi-vector DDoS attacks up to 287% more sophisticated - [Hacker Noon](https://hackernoon.com/smart-attacks-smarter-defenses-how-ai-is-transforming-ddos-attacks?source=rss)

## Atlas: Browser & Agent Mode
- Experts warn ChatGPT Atlas Agent Mode can leak emails, passwords and clipboard via prompt injection - [Fortune](https://fortune.com/2025/10/23/cybersecurity-vulnerabilities-openai-chatgpt-atlas-ai-browser-leak-user-data-malware-prompt-injection/)
- Ars Technica publishes 10-point hands-on review of ChatGPT Atlas Agent Mode - [Ars Technica](https://arstechnica.com/features/2025/10/we-let-openais-agent-mode-surf-the-web-for-us-heres-what-happened/)
- OpenAI to add profiles, tab groups, ad blocker, 1Password fixes and Drive/Excel integration to ChatGPT Atlas - [The Verge](https://www.theverge.com/news/805160/openai-chatgpt-atlas-updates)
- OpenAI launches ChatGPT Atlas to challenge Google Search, pursue monetization and India expansion - [Live Mint](https://www.livemint.com/companies/news/openai-chatgpt-atlas-ai-browser-google-search-india-google-silicon-valley-internet-ai-platform-big-tech-data-startup-11761118063947.html)
- CNET recommends DALL·E 3 in ChatGPT Plus, warns of legal risks from training data - [CNET](https://www.cnet.com/tech/services-and-software/create-the-perfect-ai-image-image-generator-tips-for-openai-google-and-canva/)
- APK teardown finds hidden ChatGPT voice mode; OpenAI moving voice into chat with live transcripts - [Android Authority](https://www.androidauthority.com/chatgpt-voice-mode-chat-integration-apk-teardown-3609526/)

## Compute: Deals & Dollars
- Google negotiating multibillion‑to‑tens‑of‑billions TPU supply deal with Anthropic - [International Business Times](https://www.ibtimes.co.uk/google-investing-billions-anthropic-move-considered-bold-response-openais-atlas-1749402)
- OpenAI, Oracle and Vantage to build \\$15B+ 'Stargate' data center in Wisconsin by 2028 - [Reuters](https://www.reuters.com/business/openai-oracle-announce-stargate-data-center-site-wisconsin-2025-10-22/)
- Analyst projects CoreWeave revenue to rise from \\$16M (2022) to \\$5.3B (2025) and \\$25B (2028) - [Benzinga](https://www.benzinga.com/markets/equities/25/10/48373546/coreweave-is-frightening-its-unbelievable-says-expert-expecting-its-revenue-to-multiply-nearly-5x-by-2028)
- Nvidia data‑center revenue rises 56% year‑over‑year to \\$41.1B - [The Motley Fool](https://www.fool.com/investing/2025/10/21/think-you-missed-the-boat-on-nvidia-heres-the-no-1/)
- Nokia shares surge as AI and cloud demand lift profit, biggest jump since 2021 - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-23/nokia-profit-surges-past-estimates-on-network-growth)
- SCHMID Group wins major orders to supply PCB production equipment for AI server boards, including a full wet-process line - [Globe Newswire](https://www.globenewswire.com/news-release/2025/10/22/3170998/0/en/SCHMID-Group-secures-Major-Orders-for-AI-Server-PCB-Production-Equipment.html)

## Gears & Racks — Hardware & Infrastructure
- Musk: Tesla AI5 chips aim for compute roughly equal to ~81,000 H100s - [CNBC](https://www.cnbc.com/2025/10/22/elon-musk-tesla-ai5-nvidia.html)
- PyTorch Monarch scales to 2,048 GPUs, adds sharded tensors and cuts recovery overheads by ≥60% - [PyTorch](https://pytorch.org/blog/introducing-pytorch-monarch/)
- Bernstein: AWS trailing AI cloud race due to GPU capacity constraints - [Business Insider](https://www.businessinsider.com/amazon-last-ai-cloud-race-why-not-death-sentence-2025-10)

## Funding & Momentum — Startups & Scale
- UnifyApps raises \\$50M Series B at ~\\$250M valuation; Ragy Thomas named chairman and co‑CEO - [Yahoo](https://finance.yahoo.com/news/ai-startup-unifyapps-raises-50-113233926.html)
- Wonder Studios, backed by OpenAI and DeepMind, raises \\$12M seed to pursue AI-generated content in 2026 - [The Wall Street Journal](https://www.wsj.com/tech/ai/openai-backed-creative-studio-raises-12-million-as-it-eyes-ai-content-drive-cad81e1f)
- Investors pivot on Tesla, favor AI-focused software, autonomy and robotics spending - [Financial Post](https://financialpost.com/commodities/energy/electric-vehicles/tesla-stops-growing-investors-pivot-to-ai)
- Ulangizi AI uses LLMs and local guidelines to provide Chichewa crop diagnostics via WhatsApp to Malawian farmers - [Rest of World](https://restofworld.org/2025/malawi-ulangizi-ai-farming-chatbot/)
- Healthcare AI Hyro raises \\$45M, says it reaches 30M patients across 45 U.S. health systems - [Tech in Asia](https://www.techinasia.com/news/us-healthcare-ai-startup-hyro-bags-45m)
- China's robovan maker Neolix raises \\$600M for R&D, eyes 2026 Hong Kong IPO - [Nikkei Asia](https://asia.nikkei.com/business/technology/chinese-robovan-startup-neolix-raises-600m-for-tech-product-development)

## Production Playbook — Enterprise AI
- Poor data strategy, not compute, is derailing enterprise visual AI; curated tooling cut review workload 80% - [Fast Company](https://www.fastcompany.com/91423778/the-hidden-data-problem-killing-enterprise-ai-projects-implementing-ai)
- Production-grade RAG: hybrid retrieval, caching, verification and routing cut latency, hallucinations, and costs - [Hacker Noon](https://hackernoon.com/10-23-2025-techbeat)
- Audit of 23 AI startups finds most are API wrappers; 91% lack vector DBs and none fine-tune models - [Hacker Noon](https://hackernoon.com/i-reverse-engineered-how-23-ai-first-companies-actually-build-their-products-and-the-tech-stack-is)
- Colab-ready enterprise assistant blueprint: FAISS retrieval, MiniLM, PII redaction, RBAC and 180-day audit logs - [MarkTechPost](https://www.marktechpost.com/2025/10/22/how-to-design-a-fully-functional-enterprise-ai-assistant-with-retrieval-augmentation-and-policy-guardrails-using-open-source-ai-models/)

## Jobs & Paychecks — Displacement and Upskilling
- AI displaces freelancers and junior developers; young developers' employment down ~20% since late 2022 - [Financial Times](https://www.ft.com/content/3d2669e3-c05e-48c9-8bb3-893c1d66de2e)
- Only 5% of US college students fully parse Bleak House opening; experts call for AI literacy and continuous upskilling - [Financial Times](https://www.ft.com/content/5e78cd00-ca0e-488d-87d0-8fef8a84f688)
- AI workplace learning moves to micro-lessons and VR simulations; Bank of America's role-play used 1.8M times, study finds AI coaches match human trainers - [Financial Times](https://www.ft.com/content/1f915aa9-c61c-4f55-b27e-47d0a0f2c016)
- AI automates hiring: bots apply to bots as resume screening accelerates; calls grow for recruiting standards - [Hacker Noon](https://hackernoon.com/when-ai-candidates-apply-to-ai-recruiters-the-end-of-human-hiring)
- Meta to cut about 600 AI-division staff despite continued AI investments - [Mashable](https://mashable.com/article/meta-lay-off-600-ai)
- PwC: AI-skilled workers earn a 56% wage premium; firms urged to redesign jobs for GenAI - [Forbes](https://www.forbes.com/sites/iese/2025/10/22/hack-yourself-how-to-find-uses-for-ai-in-everything-you-do-at-work/)

## Dev Tools Unleashed — Coding, AIOps & Agents
- Autonomous AIOps shifts SRE practices — ~40% adoption by 2024; bank auto-resolved 62% of incidents - [Hacker Noon](https://hackernoon.com/from-automation-to-autonomy-how-ai-is-transforming-site-reliability-engineering)
- Sourcegraph opens Amp to all with VS Code extension and CLI; reportedly generates 70–80% of commits - [Hacker Noon](https://hackernoon.com/sourcegraph-drops-the-waitlist-amp-its-agentic-coding-tool-opens-to-everyone)
- Sonar profiles LLM 'coding personalities' and warns they produce insecure, debt‑prone code - [Forbes](https://www.forbes.com/sites/lanceeliot/2025/10/22/the-best-way-to-vibe-code-is-to-first-know-the-ai-coding-personality-that-you-are-dealing-with/)
- Gemini CLI v0.9 adds PTY and interactive shell; now available via npm - [Google Developers Blog](https://developers.googleblog.com/en/say-hello-to-a-new-level-of-interactivity-in-gemini-cli/)

## Cloud & the Bottom Line — Microsoft Moves
- Microsoft surveys customers on Copilot for on‑prem Exchange, probing data-sending and safeguard needs - [The Register](https://www.theregister.com/2025/10/23/copilot_exchange_server/)
- Nadella: Microsoft mobilizes 34,000 engineers, 400+ Azure datacenters, 2 GW new Azure AI compute and 11,000+ AI models - [VentureBeat](https://venturebeat.com/ai/what-enterprises-can-take-away-from-microsoft-ceo-satya-nadellas-shareholder)
- Analysts: Microsoft among top AI stocks as AI features boost software revenue - [Biztoc](https://biztoc.com/x/0290144423dc727f)

## Robots on the Move — Warehouses, Delivery & Humanoids
- Amazon unveils 10 robots including Blue Jay (handles ~75% of item types), warns AI will cut jobs; plans 250,000 seasonal hires - [The Verge](https://www.theverge.com/news/805098/amazon-robots-ai-warehouses)
- Amazon pilots AR glasses to give delivery drivers hands-free scanning and navigation - [GeekWire](https://www.geekwire.com/2025/amazon-unveils-ai-powered-augmented-reality-glasses-for-delivery-drivers/)
- FIX 2025 mobility expo in Daegu spotlights EVs, VTOLs, autonomous driving and humanoid robots - [DigiTimes](https://www.digitimes.com/news/a20251022VL209/south-korea-flying-car-robotics-xpeng-2025.html)
- Musk says Tesla's Optimus could perform surgery; production starts next year with a 1M/year target - [Business Insider](https://www.businessinsider.com/elon-musk-says-optimus-will-be-an-incredible-surgeon-2025-10)

## Alibaba's Qwen Wave — Consumer & Cloud Push
- Alibaba adds Qwen3 chatbot to Quark app; Quark AI Glasses pre-sale starts Oct. 24 - [Reuters](https://www.reuters.com/world/asia-pacific/alibaba-launches-ai-chatbot-service-renewed-consumer-push-2025-10-23/) [South China Morning Post](https://www.scmp.com/tech/tech-trends/article/3330034/alibabas-quark-app-adds-ai-chatbot-challenge-bytedances-doubao-tencents-yuanbao)
- Airbnb adopts Alibaba's Qwen for customer service, citing faster, cheaper support - [Yahoo](https://finance.yahoo.com/news/airbnb-picks-alibabas-qwen-over-093000045.html)
- Alibaba debuts two visual-language models for Alibaba Cloud's multimodal AI - [Biztoc](https://biztoc.com/x/fc79d5e56ee57ccb)

## Other News
- Apple Intelligence beta brings six everyday AI features to iPhone 15 Pro, 16 and 17 models - [CNET](https://www.cnet.com/tech/services-and-software/yes-apple-intelligence-can-be-useful-here-are-6-features-i-use-regularly/)
- McKinsey warns AI could cut banks' profits by as much as \\$170 billion - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-23/mckinsey-says-bank-profits-face-possible-170-billion-ai-hit)


In [31]:
print(state.get_workflow_status_report())



WORKFLOW STATUS
Progress: 100.0% (9/9 complete)
Status Summary: 9 complete, 0 started, 0 failed, 0 not started
Status: All steps complete or started

Step Details:
  Step 0: Gather URLs: complete
    Status: ✅ Step 1 gather_urls completed successfully! Gathered 853 articles from 17 sources.

📊 Articles stored in persistent state: 735
  Step 1: Filter URLs: complete
    Status: ✅ Step 2 filter_urls completed successfully! Removed 467 duplicate URLs, classified 268 new articles, found 122 AI-related.
  Step 2: Download Articles: complete
    Status: ✅ Step 3 download_articles completed successfully! Downloaded 111 AI-related articles with 96% success rate.
📊 Average article length: 5631 characters
🔗 Content stored in persistent state.
  Step 3: Extract Summaries: complete
    Status: ✅ Step 4 extract_summaries completed successfully! Generated AI-powered summaries for 116/116 articles.
💾 Summaries stored in headline DataFrame.
  Step 4: Rate Articles: complete
    Status: ✅ Step 5 rate_a

In [32]:
state.print_workflow_status()



📊 WORKFLOW STATUS
📈 Progress: 100.0% (9/9 complete)
🎉 All steps complete!

📋 Step Details:
  ✅ Step 0: Gather URLs            | complete
      📝 Status: ✅ Step 1 gather_urls completed successfully! Gathered 853 articles from 17 sources.

📊 Articles stored in persistent state: 735
  ✅ Step 1: Filter URLs            | complete
      📝 Status: ✅ Step 2 filter_urls completed successfully! Removed 467 duplicate URLs, classified 268 new articles, found 122 AI-related.
  ✅ Step 2: Download Articles      | complete
      📝 Status: ✅ Step 3 download_articles completed successfully! Downloaded 111 AI-related articles with 96% success rate.
📊 Average article length: 5631 characters
🔗 Content stored in persistent state.
  ✅ Step 3: Extract Summaries      | complete
      📝 Status: ✅ Step 4 extract_summaries completed successfully! Generated AI-powered summaries for 116/116 articles.
💾 Summaries stored in headline DataFrame.
  ✅ Step 4: Rate Articles          | complete
      📝 Status: ✅ Step 5 ra